# Resumen clase anterior

In [5]:
import numpy as np

def PatternCount(Text,Pattern):
    count = 0
    for i in range( len(Text) - len(Pattern) +1 ):
        if Text[i:i+len(Pattern)] == Pattern:
            count += 1
    return(count)

def FrequentWords(Text, k):
    FrequentPatterns = set()
    Count = [0]*( len(Text)-k+1 )
    for i in range( len(Text) - k +1 ):
        Pattern = Text[ i:(i+k)]
        Count[i] = PatternCount(Text, Pattern) # PatternCount cuenta el número de veces 
                                               # que Pattern aparece en Text
    maxCount = np.max(Count)
    for i in range(len(Text) - k +1 ):
        if Count[i] == maxCount:
            FrequentPatterns.add( Text[i:(i+k)] )
            
    return( (FrequentPatterns),maxCount ) #,Count

Text="atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaac\
ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca\
cggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgactt\
gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt\
acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga\
tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat\
tgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaag\
atcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtt\
tccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc"
k=9

FrequentWords(Text, k)

({'atgatcaag', 'ctcttgatc', 'cttgatcat', 'tcttgatca'}, 3)

Una manera más eficiente de buscar patrones
=====================================

Ya conseguimos nuestro objetivo de crear una herramienta para encontrar los patrones más comunes. Ésta funciona bien, pero no es muy eficiente. En ejemplos reales, en donde las secuencias genéticas tienen millones de bases, nuestra función es muy lenta. Utliza un ciclo for para recorrer la secuencia completa, pero por cada iteración del ciclo for llama a la función PatternCount, la que a su vez recorre nuevamente la secuencia completa, de principio a fin. 

Una manera más eficiente de hacer los calculos es almacenara el número de veces que ser recorre la secuencia no en una lista, sino en un diccionario. En python un diccionario se define de la siguiente manera:

```python
    x={} # a nuestro diccionario lo llamaremos x
    x["secuencia1"]=1
    x["secuencia2"]=3
    x["secuencia1"] = x["secuencia1"] +1
```

Este es el seudo código para implementar esta mejora

```python
    FrequencyDicionary(Text, k)
        freqMap ← empty map
        n ← |Text|
        for i ← 0 to n − k
            Pattern ← Text(i, k)
            if freqMap[Pattern] doesn't exist
                freqMap[Pattern]← 1
            else
               freqMap[pattern] ←freqMap[pattern]+1 
        return freqMap
    
    BetterFrequentWords(Text, k)
        FrequentPatterns ← an array of strings of length 0
        freqMap ← FrequencyDictionary(Text, k)
        max ← MaxMap(freqMap)
        for all strings Pattern in freqMap
            if freqMap[pattern] = max
                append Pattern to frequentPatterns
        return frequentPatterns    
```    

## Ejercicio

Implementar las funciones ```FrequencyDicionary(Text, k)``` y ```BetterFrequentWords(Text, k)```.

# Alineamiento de secuencias genéticas



## Introducción 

Las secuencias genéticas evoluciónan a través de mutaciones, inserciones, y eliminaciones de nucleotridos. Veamos algunos ejemplos: 

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_mutations.png" alt="Drawing" style="width: 400px;"/>

### Alineamineto de nucleotidos

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_nucleotides.png" alt="Drawing" style="width: 700px;"/>

### Alineamiento de aminoacidos


<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_aaPolaridad.png" alt="Drawing" style="width: 200px;" align="left"/>

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_aminoacids.png" alt="Drawing" style="width: 620px;"/>

Modelos de medición de la similaridad de secuencias
---------------------------


\begin{eqnarray}
P(x,y|R) = \prod_i q_{x_i} \prod_j q_{y_j}
\end{eqnarray}


\begin{eqnarray}
P(x,y|M) = \prod_i p_{x_i y_i}
\end{eqnarray}

\begin{eqnarray} 
\frac{P(x,y|M)}{ P(x,y|R) } = \frac{\prod_i p_{x_i y_i} } {\prod_i q_{x_i} \prod_j q_{y_j}} = \prod_i \frac{p_{x_iy_i}}{q_{x_i}q_{y_i}}
\end{eqnarray}

\begin{eqnarray} 
&S=\sum_is(x_i,y_i) \\
\text{en donde:}& \\
&s(a,b)=log \left( \frac{p_{ab}}{q_aq_b} \right)
\end{eqnarray}

Espacios (gaps)
--------------------

\begin{eqnarray}
\gamma(g) &= -gd
\end{eqnarray}

Ejercicio 1
-------------

1. Descargar las matrices de alineamiento desde las librerías de ```biopython```, las cuales se instalan de la siguiente manera:


```pythohn
pip install biopython
```

2. Cargar las matrices BBLOSUM62 Y BLOSUM50.

In [21]:
# Ejercicio 1
from Bio.SubsMat import MatrixInfo
import pandas as pd
import numpy as np


def createList(seq):
    x=[]
    for i in seq:
        x.append(i)
    return(x)

aa="CSTPAGNDEQHRKMILVFYW" #string
aa=createList(aa)

#aa=["C","S","T","P","A","G","N","D","E","Q","H","R","K","M","I","L","V","F","Y","W"] # lista

scoreMatrix=pd.DataFrame(index=aa,columns=aa)  

for i in aa:
    for j in aa:
        pair = (i,j)
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        scoreMatrix.loc[i,j]=MatrixInfo.blosum50[pair]
scoreMatrix

,C,S,T,P,A,G,N,D,E,Q,H,R,K,M,I,L,V,F,Y,W
C,13,-1,-1,-4,-1,-3,-2,-4,-3,-3,-3,-4,-3,-2,-2,-2,-1,-2,-3,-5
S,-1,5,2,-1,1,0,1,0,-1,0,-1,-1,0,-2,-3,-3,-2,-3,-2,-4
T,-1,2,5,-1,0,-2,0,-1,-1,-1,-2,-1,-1,-1,-1,-1,0,-2,-2,-3
P,-4,-1,-1,10,-1,-2,-2,-1,-1,-1,-2,-3,-1,-3,-3,-4,-3,-4,-3,-4
A,-1,1,0,-1,5,0,-1,-2,-1,-1,-2,-2,-1,-1,-1,-2,0,-3,-2,-3
G,-3,0,-2,-2,0,8,0,-1,-3,-2,-2,-3,-2,-3,-4,-4,-4,-4,-3,-3
N,-2,1,0,-2,-1,0,7,2,0,0,1,-1,0,-2,-3,-4,-3,-4,-2,-4
D,-4,0,-1,-1,-2,-1,2,8,2,0,-1,-2,-1,-4,-4,-4,-4,-5,-3,-5
E,-3,-1,-1,-1,-1,-3,0,2,6,2,0,0,1,-2,-4,-3,-3,-3,-2,-3
Q,-3,0,-1,-1,-1,-2,0,0,2,7,1,1,2,0,-3,-2,-3,-4,-1,-1


Ejercicio 2
---------

Computa el score de cada uno de los siguientes alineamientos

1.

ACGTCCATAGGCATCGATACTGAACTGGTCCATACGATACT\
ACGTCC**C**TAGGCATCGATACTGAACTG**T**TCCATACGATACT


2.

ACGTCCATAGGCATCG**-**TACTGAACTGGTCCATACGATACT\
ACGTCC**C**TAGGCATCGATACTGAACTG**T**TCCATA**-**GATACT

3.

VFRKAGND\
V**K**RKA**A**ND

4.

VFR**-**AGND\
V**K**RKA**A**ND

Tarea optativa
------------

1. Dada las secuencas ```seq1``` y ```seq2```, escribe una función ```score(seq1,seq2)``` para calcular su score.
2. Aplica ```score(seq1,seq2)``` a los 4 alineamientos del ejercicio anterior.

NOTA: Usar $d=8$ y matriz BLOSUM50 de libre elección.